In [1]:
import os 
os.chdir("/Users/antoniamuhleck/Desktop/Hiwi/AWEWA/Python files")

## Imports

In [9]:
from creating_dust_input_files import *
from creating_elliptic_wing import create_elliptic_wing
from running_DUST_files import *
from velocity_in_front_of_kite import *
import pandas as pd # type: ignore
import numpy as np # type: ignore
import os 
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

## Defining the paths

In [10]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

In [11]:
folder_path_DUST_files = os.path.join(parent_dir, "DUST files") # path to folder with DUST files, here your dust input files will be created as well as the Output folder and Postprocessing folder
file_path_awebox = os.path.join(parent_dir, "awebox files/AAA_3DOF_N8_d4_Nrings3_Ndup3_Nfar4_rectangle_convfar.csv") # path to the awebox file of the trajoctory

## Creating the Wing

In [12]:
AR = 10
span = 44.73
N_elements = 10
nelem_span = 4
end_chord = 1.0
airfoil_profile = "NACA0012"
airfoil_table = "/Users/antoniamuhleck/Desktop/Hiwi/AWEWA/airfoils/naca0012.c81"

In [13]:
create_elliptic_wing(folder_path_DUST_files, AR, span, N_elements, nelem_span, airfoil_profile, end_chord, "vl",3, airfoil_table )

## Creating the DUST files and running the simulation

In [14]:
n_rot = 2
res = 4
single_or_dual = "dual"
particles_or_panels = "particles"
apply_pitch_correction = True
CL0 = 0
alpha_max = 12
alpha_min = 0

In [15]:
create_dust_files(n_rot, res, file_path_awebox, folder_path_DUST_files, single_or_dual, apply_pitch_correction, CL0, alpha_max, alpha_min, particles_or_panels, 80)

In [18]:
# run_full_simulation(folder_path_DUST_files)

## Additional postprocessing

### Velocity in front of the wing

In [19]:
# velocities_20m, time = velocity_in_front_of_kite(2, res, file_path_awebox, folder_path_DUST_files, single_or_dual, apply_pitch_correction, CL0, alpha_max, alpha_min, 20)

In [20]:
# plt.plot(time, np.array(velocities_20m[0])[:,0], label = "v_x")
# plt.plot(time, np.array(velocities_20m[0])[:,1], label = "v_y")
# plt.plot(time, np.array(velocities_20m[0])[:,2], label = "v_z")
# plt.legend()
# plt.title("kite 1, d = 20m, first rotation")
# plt.xlabel("time [s]")
# plt.ylabel("v[m/s]")
# plt.show()

## Visualisations

### Heatmap in kite plane

In [ ]:
file_path_hm_kite_plane = "post_heatmap_xz_0239.vtr"

In [ ]:
mesh = pv.read(file_path_hm_kite_plane)

mesh["velocity"][:, 0] -= u_inf

# calculating velocity components orthogonal to the plane
v_components_orthogonal_to_plane = v_normal @ mesh["velocity"].T

# Convert the result back to a pyvista_ndarray
v_components_orthogonal_to_plane = np.clip(v_components_orthogonal_to_plane, -10, 10)
v_components_orthogonal_to_plane = pv.pyvista_ndarray(v_components_orthogonal_to_plane)

nx, ny, nz = mesh.dimensions
velocity_grid = v_components_orthogonal_to_plane.reshape((nz, ny, nx))

# works only for nz = nx
diagonal_velocity = np.array([velocity_grid[i, :, nz - i - 1] for i in range(nz)])
X = np.linspace(mesh.bounds[2], mesh.bounds[3], diagonal_velocity.shape[1])
Y = np.linspace(mesh.bounds[4], mesh.bounds[5], diagonal_velocity.shape[0])
X, Y = np.meshgrid(X, Y)
# Normalize the colormap to center at 0
norm = mcolors.TwoSlopeNorm(vmin=diagonal_velocity.min(), vcenter=0, vmax=diagonal_velocity.max())

plt.figure(figsize=(8, 6))  # Adjust the figure size as needed
ax = plt.gca()
cset = plt.contourf(X, Y, diagonal_velocity, levels=30, cmap=plt.cm.RdBu_r, norm=norm)

# Creating the plot
# Adding a legend
cbar = plt.colorbar(cset, label='$v_\mathrm{ind}$ [m/s]')
cbar.set_label('$v_\mathrm{ind}$ [m/s]') 

# Adding labels
plt.xlabel('$e_\mathrm{1}$')
plt.ylabel('$e_\mathrm{2}$')
plt.show()

### Heatmap in the xz plane

In [15]:
plot_dict = pd.read_csv(file_path_awebox)
u_inf = np.average(plot_dict["outputs_aerodynamics_u_infty2_0"])

elevation_opt = np.arcsin(np.mean(plot_dict['x_q10_2']) / plot_dict['theta_l_t_0'][0])
v_normal = [np.cos(elevation_opt), 0, np.sin(elevation_opt)]

In [16]:
file_path_hm_xz = os.path.join(parent_dir, "DUST files/Postprocessing/post_heatmap_xz_0239.vtr")

In [21]:
# mesh = pv.read(file_path_hm_xz)

# mesh["velocity"][:, 0] -= u_inf
# v_components_orthogonal_to_plane = v_normal @ mesh["velocity"].T

# x, z = mesh.points[:, 0], mesh.points[:, 2]  

# nx, ny = len(np.unique(x)), len(np.unique(z))
# grid_values = v_components_orthogonal_to_plane.reshape((ny, nx)) 

# grid_values = np.clip(grid_values, -12, 6)

# norm = mcolors.TwoSlopeNorm(vmin=grid_values.min(), vcenter=0, vmax=grid_values.max())

# X = np.linspace(mesh.bounds[0], mesh.bounds[1], grid_values.shape[1])
# Y = np.linspace(mesh.bounds[4], mesh.bounds[5], grid_values.shape[0])
# X, Y = np.meshgrid(X, Y)



# plt.figure(figsize=(10, 6))  
# ax = plt.gca()
# cset = plt.contourf(X, Y, grid_values, levels=30, cmap=plt.cm.RdBu_r, norm=norm)

# cbar = plt.colorbar(cset, label='$v_\mathrm{ind}$ [m/s]')
# cbar.set_label('$v_\mathrm{ind}$ [m/s]') 

# plt.xlabel('x')
# plt.ylabel('z')

# plt.show()
